In [ ]:
from pathlib import Path
import importlib
import warnings
import time
import os
import polars as pl
from datetime import datetime
import time
warnings.filterwarnings("ignore")  # Oculta todos los warnings

# Mis modulos
import process_data as dp
import transform_data as td
import html_generator as hg
import send_email as sm
import show_html as sh

In [42]:
# import pyfiglet
# print(pyfiglet.figlet_format("Gestor de Vacaciones"))

banner = r"""
  ____           _                   _      
 / ___| ___  ___| |_ ___  _ __    __| | ___ 
| |  _ / _ \/ __| __/ _ \| '__|  / _` |/ _ \
| |_| |  __/\__ \ || (_) | |    | (_| |  __/
 \____|\___||___/\__\___/|_|     \__,_|\___|
                                            
__     __                   _                       
\ \   / /_ _  ___ __ _  ___(_) ___  _ __   ___  ___ 
 \ \ / / _` |/ __/ _` |/ __| |/ _ \| '_ \ / _ \/ __|
  \ V / (_| | (_| (_| | (__| | (_) | | | |  __/\__ \
   \_/ \__,_|\___\__,_|\___|_|\___/|_| |_|\___||___/

    💥 VISUALIZADOR Y EDITOR DE VACACIONES 💥
"""

print(banner, end='\n\n')
time.sleep(1)


  ____           _                   _      
 / ___| ___  ___| |_ ___  _ __    __| | ___ 
| |  _ / _ \/ __| __/ _ \| '__|  / _` |/ _ \
| |_| |  __/\__ \ || (_) | |    | (_| |  __/
 \____|\___||___/\__\___/|_|     \__,_|\___|

__     __                   _                       
\ \   / /_ _  ___ __ _  ___(_) ___  _ __   ___  ___ 
 \ \ / / _` |/ __/ _` |/ __| |/ _ \| '_ \ / _ \/ __|
  \ V / (_| | (_| (_| | (__| | (_) | | | |  __/\__ \
   \_/ \__,_|\___\__,_|\___|_|\___/|_| |_|\___||___/

    💥 VISUALIZADOR Y EDITOR DE VACACIONES 💥




In [ ]:
# Constantes
this_year = datetime.today().date().year

# Diccionarios
months = {
    1: 'Enero', 2: 'Febrero', 3: 'Marzo', 4: 'Abril', 5: 'Mayo', 6: 'Junio',
    7: 'Julio', 8: 'Agosto', 9: 'Septiembre', 10: 'Octubre', 
    11: 'Noviembre', 12: 'Diciembre'
}
vacaciones = {
    "name": "Gestor de Vacaciones",
    "file_name_xlsx": "DATOS_VACACIONES_2020-2025.xlsx",
    "file_name_csv": "vacaciones_data.csv",
    "sheet_name": "DATOS",
    "relevant_columns": [
        'APELLIDOS',
        'NOMBRES',
        'DNI',
        'CARGO',
        'Fecha Ingreso',
        'Vacaciones 2020-2021',
        'Vacaciones 2021-2022',
        'Vacaciones 2022-2023',
        'Vacaciones 2023-2024',
        'Vacaciones 2024-2025',
    ],
    "new_columns": [
        "DIAS_ACUMULADOS",
        "VACACIONES_GOZADAS",
        "VACACION_GOZADA_ACTUAL", # 3 estados (No vacaciono, esta vacacionando, por vacacionar)
        "VACACIONES_ACUMULADAS", # Dias de vacaciones acumuladas que tiene pendiente para gozar
    ]
}
VACACION_GOZADA_ACTUAL_ESTADOS = {
    0: f"Vacaciones no aplicables",
    1: f"Vacaciones {this_year} gozadas",
    2: "Gozando vacaciones actualmente",
    3: f"Vacaciones {this_year} por gozar",
}

# Listas

# Constantes
project_address = r"C:\Informacion\Otros\RRHH\Automatizacion\vacation_manager"
CONSOLIDADO = 'consolidado.html'
CONSOLIDADO_SUBJECT = 'Reporte consolidado de vacaciones'
PERSONAL = 'personal.html'
PERSONAL_SUBJECT = f'Reporte de vacaciones personal'
VACACION = 'vacacion.html'
VACACION_SUBJECT = 'ALERTA: Detección de vacaciones a gozar'
ANIVERSARIO = 'aniversario.html'
ANIVERSARIO_SUBJECT = 'ALERTA: Detección de aniversario'

MAIL_TO = "rrhh@ayacda.com; gportocarrerob@unsa.edu.pe;"
#MAIL_TO = "gportocarrerob@unsa.edu.pe; ainformacion@ayacda.com"
MAIL_CC = ";"

### Procesamiento de datos

In [44]:
importlib.reload(dp)

df = dp.main(project_address, vacaciones)

### Transformacion de datos

In [ ]:
importlib.reload(td)

df = td.main(df)

shape: (66, 21)
┌───────────┬───────────┬──────────┬───────────┬───┬───────────┬───────────┬───────────┬───────────┐
│ APELLIDOS ┆ NOMBRES   ┆ DNI      ┆ CARGO     ┆ … ┆ ANIVERSAR ┆ PROXIMO_A ┆ DIAS_PARA ┆ ALERTA_AN │
│ ---       ┆ ---       ┆ ---      ┆ ---       ┆   ┆ IO_TEMP   ┆ NIVERSARI ┆ _ANIVERSA ┆ IVERSARIO │
│ str       ┆ str       ┆ i64      ┆ str       ┆   ┆ ---       ┆ O         ┆ RIO       ┆ ---       │
│           ┆           ┆          ┆           ┆   ┆ datetime[ ┆ ---       ┆ ---       ┆ str       │
│           ┆           ┆          ┆           ┆   ┆ μs]       ┆ datetime[ ┆ i64       ┆           │
│           ┆           ┆          ┆           ┆   ┆           ┆ μs]       ┆           ┆           │
╞═══════════╪═══════════╪══════════╪═══════════╪═══╪═══════════╪═══════════╪═══════════╪═══════════╡
│ ALVARES   ┆ EDWIN     ┆ 70580389 ┆ CHOFER    ┆ … ┆ 2025-03-0 ┆ 2026-03-0 ┆ 260       ┆ > 1       │
│ HUAMAN    ┆           ┆          ┆ VENDEDOR  ┆   ┆ 7         ┆ 7         

### Generacion de reportes

In [ ]:
importlib.reload(hg)
importlib.reload(sm)
importlib.reload(sm)
importlib.reload(sh)

def send_reports(option, group_option):
    hg.main(
            project_address, 
            df, 
            months, 
            VACACION_GOZADA_ACTUAL_ESTADOS,
            CONSOLIDADO,
            PERSONAL, 
            VACACION, 
            ANIVERSARIO,
            group_option
        )
    
    if option == 1:
        sh.main(project_address, CONSOLIDADO)
        respuesta = input("\n>> Enviar reporte al correo (si/no): ")        
        if respuesta == "si" or respuesta == "s":
            sm.main(project_address, MAIL_TO, MAIL_CC, CONSOLIDADO, CONSOLIDADO_SUBJECT)
    
    elif option == 2:
        sh.main(project_address, PERSONAL)
        respuesta = input("\n>> Enviar reporte al correo (si/no): ")        
        if respuesta == "si" or respuesta == "s":
            sm.main(project_address, MAIL_TO, MAIL_CC, PERSONAL, PERSONAL_SUBJECT)

    elif option == 3:
        sh.main(project_address, VACACION)
        respuesta = input("\n>> Enviar alerta de vacaciones al correo (si/no): ")
        if respuesta == "si" or respuesta == "s":
            sm.main(project_address, MAIL_TO, MAIL_CC, VACACION, VACACION_SUBJECT)

    elif option == 4:
        sh.main(project_address, ANIVERSARIO)
        respuesta = input("\n>> Enviar alerta de aniversario al correo (si/no): ")
        if respuesta == "si" or respuesta == "s":
            sm.main(project_address, MAIL_TO, MAIL_CC, ANIVERSARIO, ANIVERSARIO_SUBJECT)

    else:
        print('[X] Opcion incorrecta [Introduza opcion valida]\n')



# 🔁 Bucle de confirmación
while True:
    # Seleccionar grupo de wssp
    print("\nSelecciona grupo de wssp:")
    print("  [1] Consolidado")
    print("  [2] Reporte personal")
    print("  [3] Alerta Vacaciones")
    print("  [4] Alerta Aniversario")
    print("  [5] Terminar proceso")

    group_option = int(input("\n>> Opción (1 - 5): "))

    if group_option == 5 or group_option > 4:
        print("\n[✓] Proceso finalizado.\n")
        break  # salir del bucle
    else:        
        send_reports(group_option, group_option)

### Limpiar carpeta

In [ ]:
carpeta = Path(project_address)
#print(list(carpeta.glob('*.png')))

# Eliminar archivos .html
for imagen in carpeta.glob('*.html'):
    imagen.unlink()  # .unlink() elimina el archivo

### Export it as .py

In [67]:
import nbformat
from nbconvert import PythonExporter

# Cargar el notebook
notebook_path = "main.ipynb"
with open(notebook_path, "r", encoding="utf-8") as f:
    nb = nbformat.read(f, as_version=4)

# Eliminar la última celda
nb['cells'] = nb['cells'][:-1]

# Convertir a script .py
exporter = PythonExporter()
codigo_python, _ = exporter.from_notebook_node(nb)

# Guardar el archivo .py
with open("main.py", "w", encoding="utf-8") as f:
    f.write(codigo_python)

print("✅ Archivo .py creado sin la última celda.")

✅ Archivo .py creado sin la última celda.
